In [1]:
import pandas as pd
import time
import sqlite3
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from langchain.vectorstores import FAISS

conn=sqlite3.connect('tem.db')

def load_db():
    print("start time:",time.ctime())
    conn=sqlite3.connect('tem.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    tables=[i[0] for i in tables]
    print(tables)
    return tables

In [2]:
from googletrans import Translator
ids=[]
dataframes=[]
meta_data=pd.DataFrame(columns=['object identifier'])

In [3]:
for i in load_db():
    df=pd.read_sql_query("SELECT * FROM "+i,conn)
    
    drop=[i for i in df.columns if 'noname' in i]
    df.drop(drop,axis=1,inplace=True)
    columns=[Translator().translate(i,dest='en').text for i in df.columns]
    # df.columns=columns
    if 'object identifier' not in columns:
        continue
    df=df.rename(columns={i:j for i,j in zip(df.columns,columns)})
    df['object identifier']=df['object identifier'].apply(lambda x:x.replace(' ','') if x else x)
    dataframes.append(df)
    ids=ids+df['object identifier'].tolist()
ids=list(set(ids))



start time: Wed Sep 20 10:11:49 2023
['building_code', 'apartment_information', 'apartment_price', 'apartment_property']


In [4]:
meta_data['object identifier']=ids
for i in dataframes:
    meta_data=pd.merge(meta_data,i,on='object identifier',how='left')
meta_data.rename(columns={'daily trading':'price'},inplace=True)
meta_data.drop_duplicates(subset=['object identifier'],inplace=True,keep='last')
meta_data.dropna(subset=['Basic road address'],inplace=True)



In [36]:
meta_data.to_csv('docs/full.csv',index=False,encoding='utf-8-sig')
filter_data=meta_data[['Basic road address','Market price closing date','Minimum selling price','Half transaction price every day','Sale limit','Is it the lowest price for jeonse?','Jeonse general transaction price','Is it all over the world?']]

In [ ]:
filter_data.to_csv('docs/filter.csv',index=False,encoding='utf-8-sig')

In [53]:
from langchain.document_loaders.csv_loader import CSVLoader
loader=CSVLoader('docs/filter.csv')
data=loader.load()

In [54]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(data, embeddings)

In [55]:
db.save_local('docs/faiss2.db')

In [56]:
db.similarity_search('서울특별시 강남구 도곡로57길 12', k=3)

[Document(page_content='\ufeffBasic road address: 서울특별시 강남구 도곡로57길 12\nMarket price closing date: 20230915\nMinimum selling price: 222500\nHalf transaction price every day: 230000\nSale limit: 237500\nIs it the lowest price for jeonse?: 130000\nJeonse general transaction price: 135000\nIs it all over the world?: 140000', metadata={'source': 'docs/filter.csv', 'row': 17364}),
 Document(page_content='\ufeffBasic road address: 서울특별시 강남구 도곡로93길 12\nMarket price closing date: 20230915\nMinimum selling price: 285000\nHalf transaction price every day: 295000\nSale limit: 305000\nIs it the lowest price for jeonse?: 205000\nJeonse general transaction price: 215000\nIs it all over the world?: 225000', metadata={'source': 'docs/filter.csv', 'row': 19063}),
 Document(page_content='\ufeffBasic road address: 서울특별시 강남구 도곡로14길 12\nMarket price closing date: 20230915\nMinimum selling price: 187500\nHalf transaction price every day: 192500\nSale limit: 197500\nIs it the lowest price for jeonse?: 92500\n

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
from langchain.prompts.prompt import PromptTemplate


custom_template = """given the following information is about apartments data your task is to \
analyze the documents:{documents} 
and answer the following questions. \
question:{question} \,

you can also get some information from chat history:{chat_history} \,
answer:"""
CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)

In [81]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
embeddings = OpenAIEmbeddings()

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    model,
    db.as_retriever(searchtype="similarity_search", k=5),
    memory=memory)

In [83]:
query = "인천광역시 남동구 에코중앙로 163 가격은 어떻습니까?"
result = qa({"question": query})    
print(result['answer'])

인천광역시 남동구 에코중앙로 163의 가격은 다음과 같습니다:
- 최소 판매 가격: 60000
- 매일 절반 거래 가격: 64000
- 판매 제한 가격: 67000
- 전세 최저 가격인가요?: 44000
- 전세 일반 거래 가격: 46000
- 전세 가격이 전세 시장 가격인가요?: 48500


In [65]:
db.similarity_search('경기도 안양시 동안구 시민대로327번길 55', k=1)

[Document(page_content='\ufeffBasic road address: 경기도 안양시 동안구 시민대로327번길 55\nMarket price closing date: 20230915\nMinimum selling price: 150000\nHalf transaction price every day: 155000\nSale limit: 160000\nIs it the lowest price for jeonse?: 85000\nJeonse general transaction price: 89000\nIs it all over the world?: 92500', metadata={'source': 'docs/filter.csv', 'row': 6})]

try with agent

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    db.as_retriever(),
    "SearchPrice",
    "Searches prices and information about apartments and return consice results."
)
tools = [tool]

In [ ]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)

In [ ]:
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [ ]:
result = agent_executor({"input": "이 주소의 개체 식별자는 무엇입니까:서울특별시 강남구 도곡로57길 12"})
result['output']

In [ ]:
result['output']

In [22]:
from langchain.llms import OpenAI   
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

from langchain.agents import create_csv_agent

In [47]:
df=pd.read_csv('docs/filter.csv',encoding='utf-8-sig')
agent = create_csv_agent(
    OpenAI(temperature=0),
    "docs/filter.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [49]:
agent.run("충청남도 공주시관골2길 24-29 가격은 얼마인가요?")



> Entering new AgentExecutor chain...
Thought: 내가 찾고자 하는 값이 데이터프레임에 있는지 확인해야 한다.
Action: python_repl_ast
Action Input: df[df['Basic road address'] == '충청남도 공주시관골2길 24-29']
Observation: Empty DataFrame
Columns: [Basic road address, Market price closing date, Minimum selling price, Half transaction price every day, Sale limit, Is it the lowest price for jeonse?, Jeonse general transaction price, Is it all over the world?]
Index: []
Thought: 데이터프레임에 없으므로 이 값은 없는 것 같다.
Final Answer: 충청남도 공주시관골2길 24-29 가격은 없습니다.

> Finished chain.


'충청남도 공주시관골2길 24-29 가격은 없습니다.'